**Install library**

In [ ]:
!pip install ultralytics pyngrok nest-asyncio fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.2 MB/s eta 0:00:00


**Connect to Drive**

In [ ]:
from google.colab import drive, userdata
import os
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


**Run System**

In [ ]:
%%writefile app.py
import uvicorn
from fastapi import FastAPI, UploadFile, File
from ultralytics import YOLO
import io
from PIL import Image, ImageDraw, ImageFont
import nest_asyncio

import torch
import numpy as np
from ultralytics.utils.ops import xyxy2xywh
from ultralytics.utils.metrics import box_iou

import base64



def process_and_merge_detections(results, iou_threshold=0.45):
    """
    1. Gộp 6 class gốc thành 3 class.
    2. Lọc các bbox đè chéo nhau (Agnostic NMS) dựa trên confidence cao nhất.
    """
    class_mapping = {
        0: 0, 1: 0,
        2: 1, 3: 1,
        4: 2, 5: 2
    }

    processed_boxes = []

    boxes = results[0].boxes.data.cpu() # [x1, y1, x2, y2, conf, cls]

    if len(boxes) == 0:
        return []

    for box in boxes:
        x1, y1, x2, y2, conf, cls_id = box.tolist()
        new_cls_id = class_mapping.get(int(cls_id), int(cls_id))
        processed_boxes.append([x1, y1, x2, y2, conf, new_cls_id])

    processed_boxes = torch.tensor(processed_boxes)

    conf_scores = processed_boxes[:, 4]
    sorted_indices = torch.argsort(conf_scores, descending=True)
    processed_boxes = processed_boxes[sorted_indices]

    keep_boxes = []
    while len(processed_boxes) > 0:
        best_box = processed_boxes[0]
        keep_boxes.append(best_box.tolist())

        if len(processed_boxes) == 1:
            break

        ious = box_iou(best_box[:4].unsqueeze(0), processed_boxes[1:][:, :4]).squeeze(0)

        remaining_indices = torch.where(ious < iou_threshold)[0]

        processed_boxes = processed_boxes[remaining_indices + 1]

    return keep_boxes

app = FastAPI()

model_path = "/content/drive/MyDrive/Food_Project_Backup/Final/ver_6/best/original/left_out_cha_mieng_fix_cha_ca/background_v5/model_new_hyper_1024_imgsz_16_batch.pt"

model = YOLO(model_path)

NEW_CLASS_NAMES = {
    0: "Suon",
    1: "Cha Ca",
    2: "Tofu"
}

COLORS = {0: "red", 1: "green", 2: "blue"}

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        image = Image.open(io.BytesIO(contents)).convert("RGB")

        results = model.predict(image, conf=0.25)
        final_boxes = process_and_merge_detections(results, iou_threshold=0.45)

        formatted_results = []
        draw = ImageDraw.Draw(image)
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf", 20)
        except:
            font = ImageFont.load_default()

        for box in final_boxes:
            x1, y1, x2, y2, conf, cls_id = box
            name = NEW_CLASS_NAMES.get(int(cls_id), "Unknown")
            color = COLORS.get(int(cls_id), "yellow")

            formatted_results.append({
                "class_id": int(cls_id),
                "class_name": name,
                "confidence": round(float(conf), 4),
                "bbox": [round(x1, 2), round(y1, 2), round(x2, 2), round(y2, 2)]
            })

            draw.rectangle([x1, y1, x2, y2], outline=color, width=4)
            draw.text((x1, y1 - 25), f"{name} {conf:.2f}", fill=color, font=font)

        buffered = io.BytesIO()
        image.save(buffered, format="JPEG")
        img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

        return {
            "status": "success",
            "detections": formatted_results,
            "image_base64": img_str
        }

    except Exception as e:
        return {"error": str(e)}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

Writing app.py


In [ ]:
from pyngrok import ngrok
import os
import time

!fuser -k 8000/tcp
ngrok.kill()

NGROK_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(NGROK_TOKEN)

public_url = ngrok.connect(8000)
print(f"\n🚀 API URL: {public_url}/predict")
print("💡 Header bắt buộc trong Postman: 'ngrok-skip-browser-warning: true'\n")

!python app.py


🚀 API URL: NgrokTunnel: "https://unchamfered-unlanterned-ignacia.ngrok-free.dev" -> "http://localhost:8000"/predict
💡 Header bắt buộc trong Postman: 'ngrok-skip-browser-warning: true'

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
INFO:     Started server process [1716]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)

0: 512x1024 (no detections), 102.9ms
Speed: 28.0ms preprocess, 102.9ms inference, 6.1ms postprocess per image at shape (1, 3, 512, 1024)
INFO:     2402:9d80:412:5cf4:fe53:9f34:52aa:6bdd:0 - "POST /predict HTTP/1.1" 200 OK

0: 1024x512 (no detections), 59.8ms
Speed: 4.1ms preprocess, 59.8ms inference, 0.7ms

In [ ]:
!tail -n 50 server.log

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
INFO:     Started server process [1621]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [ ]:
!lsof -i:8000

In [ ]:
!fuser -k 8000/tcp

8000/tcp:             1621


In [ ]:
!python app.py

INFO:     Started server process [2282]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)

0: 96x640 (no detections), 92.1ms
Speed: 23.9ms preprocess, 92.1ms inference, 5.1ms postprocess per image at shape (1, 3, 96, 640)
INFO:     1.52.83.0:0 - "POST /predict HTTP/1.1" 200 OK

0: 352x640 1 suon_cot_let, 3 cha_cat_lats, 3 cha_miengs, 3 tofu_chiens, 1 tofu_trang, 59.1ms
Speed: 2.5ms preprocess, 59.1ms inference, 46.9ms postprocess per image at shape (1, 3, 352, 640)
INFO:     1.52.83.0:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2282]
